In [1]:
import json

In [12]:
f = open("US_category_id.json",)

In [13]:
data = json.load(f)

In [14]:
categoryMap = {}
for i in data["items"]:
    categoryMap[i["id"]]  = i["snippet"]["title"]

In [15]:
import pandas as pd

In [43]:
df = pd.read_csv('USvideos.csv', usecols=['title','title','channel_title', 'tags', 'description','category_id'])
df.drop_duplicates(subset=["title","channel_title","tags","description"], inplace=True)

In [44]:
num_rows = df.count()[0]

In [45]:
videoData = []
for i in range(num_rows):
    current_vid = df.iloc[i]
    row = []
    channel_title = preProcess(current_vid["channel_title"], 0)
    title = preProcess(current_vid["title"], 4)
    tags = preProcess(current_vid["tags"], 4)
    description = preProcess(current_vid["description"], 4)
    category = categoryMap[str(current_vid["category_id"])]
    
    row.append(channel_title)
    row.append(title)
    row.append(tags)
    row.append(description)
    row.append(category)
    videoData.append(row)


In [46]:
import re

def preProcess(data, length):
    data =  re.sub('[^a-zA-Z0-9 \n\...]', ' ', str(data))
    string_encode = str(data).encode("ascii", "ignore")
    string_decode = string_encode.decode()
    data = string_decode
    dataArr = str(data).strip().split(" ")
    for i in  reversed(range(len(dataArr))):
        dataArr[i] = str(dataArr[i]).lower()
        if len(dataArr[i]) == 0:
            del dataArr[i]
        elif len(dataArr[i]) < length:
            del dataArr[i]
    dataArr = ";".join(dataArr)
    return dataArr

In [47]:
import csv  

header = ['Video Title', 'Channel Title', 'Tags', 'Description', 'Category']

with open('video_data.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)
    writer.writerows(videoData)

In [1]:
###############################

In [131]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import f1_score

file = 'video_data.csv'
data = pd.read_csv(file)

In [132]:
data["All"] = data["Tags"] + ";" + data["Video Title"] + ";" +  data["Description"] + ";" +  data["Channel Title"]

In [133]:
vectorizer = CountVectorizer(stop_words="english")

In [134]:
all_features = vectorizer.fit_transform(data['All'].fillna(' '))

In [135]:
X_train, X_test, y_train, y_test = train_test_split(all_features, data["Category"], test_size=0.3, random_state=7)

In [136]:
classifier = MultinomialNB()

In [137]:
classifier.fit(X_train, y_train)

MultinomialNB()

In [138]:
correct = (y_test == classifier.predict(X_test)).sum()

In [139]:
incorrect = y_test.size - correct

In [140]:
accuracy = incorrect / (correct + incorrect)
accuracy = 1 - accuracy

In [141]:
f1 = f1_score(y_test, classifier.predict(X_test),average='weighted')

In [142]:
print("Accuracy: " + str(accuracy))
print("F1 Score: " + str(f1))

Accuracy: 0.7914951989026063
F1 Score: 0.7831375359263314


In [143]:
#########################################

In [144]:
def checkCategory(example):
    emptyArr = []
    emptyArr.append(example)
    doc_term_matrix = vectorizer.transform(emptyArr)
    classifier.predict(doc_term_matrix)
    class_prob_list = classifier.predict_proba(doc_term_matrix)[0]
    map = {}
    for i in range(len(class_prob_list)):
        map[str(classifier.classes_[i])] = class_prob_list[i]
    sort_classes = sorted(map.items(), key=lambda x: x[1], reverse=True)
    
    print(example + "is most likely category: " + str(sort_classes[0]))
    print(" ")
    print("Probabilities")
    for i in sort_classes:
        print(i[0], i[1])

In [129]:
checkCategory("Lebron")

Lebronis most likely category: ('Sports', 0.6977095931353297)
 
Probabilities
Sports 0.6977095931353297
Entertainment 0.16183396865491845
People & Blogs 0.0668531460966478
Comedy 0.027118780866376844
News & Politics 0.016775116346600498
Music 0.007527671194691132
Howto & Style 0.005113078972453804
Science & Technology 0.00443513399759504
Film & Animation 0.0036876458963477935
Education 0.0030033012953389285
Pets & Animals 0.0019416398858813266
Gaming 0.0016659108693151325
Autos & Vehicles 0.0010459914971282197
Travel & Events 0.0008455590535212425
Nonprofits & Activism 0.0002517910010789659
Shows 0.00019167123677504667
